In [ ]:
import numpy as np
import math
from sklearn.metrics import accuracy_score 
from tqdm import tqdm
from tensorflow import keras
import tensorflow as tf
from keras.datasets import cifar10  #to import dataset

ImportError: cannot import name 'get_config' from 'tensorflow.python.eager.context' (/Users/adityadandwate/opt/anaconda3/envs/tf/lib/python3.7/site-packages/tensorflow_core/python/eager/context.py)

In [2]:
tf.test.gpu_device_name()
#testing for gpu

2022-07-29 06:05:37.583105: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-29 06:05:37.643639: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-29 06:05:37.753387: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-29 06:05:37.754160: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

'/device:GPU:0'

In [3]:
#If running in kaggle

# import os
# for dirname, _, filenames in os.walk('/kaggle/'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

#else use this
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

/kaggle/src/script.ipynb
/kaggle/lib/kaggle/gcp.py
/kaggle/input/cifar10kerasfilescifar10loadingdata/cifar-10.npz
/kaggle/input/cifar10-python/cifar-10-python.tar.gz
/kaggle/input/cifar10-python/cifar-10-batches-py/data_batch_1
/kaggle/input/cifar10-python/cifar-10-batches-py/data_batch_2
/kaggle/input/cifar10-python/cifar-10-batches-py/batches.meta
/kaggle/input/cifar10-python/cifar-10-batches-py/test_batch
/kaggle/input/cifar10-python/cifar-10-batches-py/data_batch_3
/kaggle/input/cifar10-python/cifar-10-batches-py/data_batch_5
/kaggle/input/cifar10-python/cifar-10-batches-py/data_batch_4
/kaggle/input/cifar10-python/cifar-10-batches-py/readme.html
/kaggle/input/fashionmnist/t10k-labels-idx1-ubyte
/kaggle/input/fashionmnist/t10k-images-idx3-ubyte
/kaggle/input/fashionmnist/fashion-mnist_test.csv
/kaggle/input/fashionmnist/fashion-mnist_train.csv
/kaggle/input/fashionmnist/train-labels-idx1-ubyte
/kaggle/input/fashionmnist/train-images-idx3-ubyte
/kaggle/working/__notebook__.ipynb


In [4]:
#------------for kaggle again---------------#

# import numpy as np
# data = np.load("/kaggle/input/cifar10kerasfilescifar10loadingdata/cifar-10.npz")
# filenames = ["x_train","y_train","x_test","y_test"]
# nps = []
# for filename in filenames:
#     nps.append(data[filename])
# x_train,y_train,x_test,y_test = nps

#-------------------------------------------#

print(x_train.shape)
print(y_train.shape)

(50000, 32, 32, 3)
(50000, 1)


In [5]:
x_train = x_train.astype('float32')
x_train = x_train/255
x_test = x_test.astype('float32')
x_test = x_test/255

In [6]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from keras.utils import np_utils


In [7]:
y_train = np_utils.to_categorical(y_train) 
y_test = np_utils.to_categorical(y_test) 
num_classes = y_test.shape[1]


In [8]:
class CifarClass():

  def __init__(self, learning_rate = 0.01):
    #self.i = 0
    self.training_images = None
    self.training_labels = None 
    self.testing_images = None
    self.testing_labels = None
    self.__learning_rate = learning_rate

  def setup_of_images(self):
    self.training_images = x_train
    train_len = len(self.training_images)
    self.training_labels = y_train

    self.testing_images = x_test
    test_len = len(self.testing_images)
    self.testing_labels = y_test

  def __ReLU(self, Z):
    return np.maximum(Z,0)

  def __ReLU_derivative(self, z):
    return z>0

  def __softmax_score(self, x):
    #print('output:', x)
    exponent = np.exp(x - np.max(x))
    #print('exponent1:', exponent)
    # print('softmax score',exponent/exponent.sum(axis=0))
    #print('exponent2:', exponent)
    return exponent/exponent.sum()

  def __forward_pass(self, X):
    
    self.__output1 = X.dot(self.__weights_1) + self.__bias_1      #shape: (1, 1024)
    #print('first_output', self.__output1)      
    self.__output1_relu = self.__ReLU(self.__output1)               #shape: (1, 1024)
    #print('relu output', self.__output1_relu)
    self.__output2 = self.__output1_relu.dot(self.__weights_2) + self.__bias_2  #shape: (10, 1)
    #print('second_output', self.__output2)
    self.__output2_softmax = self.__softmax_score(self.__output2)               #shape: (10, 1)
    #print('softmax output', self.__output2_softmax)

  def __backward_propogation(self, cross_entropy_gradient_2, difference_total_2, difference_total_1, cross_entropy_gradient_1):

    #updating weights_2
    step_size_2 = (cross_entropy_gradient_2*self.__learning_rate).T     #shape: (1024, 10)
    self.__weights_2 = self.__weights_2 - step_size_2

    #updating bias_2
    self.__bias_2 = self.__bias_2 - (difference_total_2)/len(self.__x_batch)    #(1,10)
    

    #updating weights_1
    step_size_1 = (cross_entropy_gradient_1*self.__learning_rate)     #(3072, 1024)
    self.__weights_1 = self.__weights_1 - step_size_1

    #updating bias_1
    self.__bias_1 = self.__bias_1 - (difference_total_1.T)/len(self.__x_batch)   #(1,1024)

  def train(self, X, y):

    self.__class_count = 10
    self.__weights_1 = np.random.rand(3072, 1024) - 0.5
    self.__weights_2 = np.random.rand(1024, 10) - 0.5
    self.__bias_1 = np.random.rand(1, 1024) - 0.5
    self.__bias_2 = np.random.rand(1,10) - 0.5
    
    print('initial b_2:', self.__bias_2)
    print('initial w_2:', self.__weights_2)

    self.__num_epochs = 25

    for i in range(self.__num_epochs):
      print('epoch idx:', i)
      for j in range(3125):
        
        self.__x_batch = X[j]                     #loading batches
        self.__y_batch = y[j]

        cross_entropy_sum_2 = 0
        cross_entropy_sum_1 = 0
        difference_total_1 = 0
        difference_total_2 = 0
        
        for k in range(16):                       #loading images for forward pass
          
          self.__forward_pass(self.__x_batch[k])   #k is image index(sanity check)
          cross_entropy_sum_2 += ((self.__output2_softmax - self.__y_batch[k]).T).dot(self.__output1_relu)  #shape: (10,1024)
          #print('shape of label is:', self.__y_batch[k].shape)     
          difference_total_2 += self.__output2_softmax - self.__y_batch[k]  #(1,10)
          #print('shape of X is:', self.__x_batch[k].shape)
          temp = self.__x_batch[k].reshape(3072,1)
          #print('shape of reshaped X is:', temp.shape)
          #print('shape of __ is:', (self.__weights_2.dot((self.__output2_softmax - self.__y_batch[k]).T)).shape)
          cross_entropy_sum_1 += (temp).dot((self.__weights_2.dot((self.__output2_softmax - self.__y_batch[k]).T).T)*self.__ReLU_derivative(self.__output1)) #(3072, 1024)
          difference_total_1 += self.__weights_2.dot((self.__output2_softmax - self.__y_batch[k]).T)            #(1024,1)

        cross_entropy_gradient_2 = cross_entropy_sum_2/len(self.__x_batch)
        cross_entropy_gradient_1 = cross_entropy_sum_1/len(self.__x_batch)

        #backward pass for the batch
        self.__backward_propogation(cross_entropy_gradient_2, difference_total_2, difference_total_1, cross_entropy_gradient_1)

      predict = self.predict(cc.testing_images)
      print('accuracy for epoch idx', i, 'is:', accuracy_score(cc.testing_labels, predict))
        #print('shapes after backward propogation:', 'w1:', self.__weights_1.shape, 'b1:', self.__bias_1.shape, 'w2:', self.__weights_2.shape, 'b2:', self.__bias_2.shape)
        #print('weights and biases after batch:', 'w1', self.__weights_1, 'b1', self.__bias_1)

      # print('final w2:', self.__weights_2)
      # print('final b2:', self.__bias_2)

  def predict(self, X):
    y= np.zeros((len(X), self.__class_count))
    for i in range(len(X)):
      self.__forward_pass(X[i])
      max_score_idx = np.argmax(self.__output2_softmax)
      y[i][max_score_idx] = 1
    return y

  
  
    

In [9]:
cc = CifarClass() 
cc.setup_of_images()

print (cc.training_images.shape)
print(cc.testing_images.shape)


(50000, 32, 32, 3)
(10000, 32, 32, 3)


In [10]:
#flattening

cc.training_images = cc.training_images.reshape(50000, 3072)

cc.training_images = cc.training_images - 0.5     

print(cc.training_images.shape)
cc.training_images = cc.training_images.reshape(3125, 16, 3072)
print('final training image shape:', cc.training_images.shape)
cc.training_labels = cc.training_labels.reshape(3125, 16, 10)         
print('final training labels shape', cc.training_labels.shape)
cc.testing_images = cc.testing_images.reshape(10000, 3072)
cc.testing_images = cc.testing_images - 0.5
print('final testing images shape', cc.testing_images.shape)

print(cc.training_labels[23][15])

#more testing
temp = cc.training_labels[23]
print(temp[15].shape)
print(temp[15][2])

(50000, 3072)
final training image shape: (3125, 16, 3072)
final training labels shape (3125, 16, 10)
final testing images shape (10000, 3072)
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
(10,)
1.0


In [11]:
def main():
  X_train = cc.training_images
  y_train = cc.training_labels

  X_test = cc.testing_images
  y_test = cc.testing_labels

  cc.train(X_train, y_train)

  global predictions 
  predictions = cc.predict(X_test)

  print('final accuracy', accuracy_score(y_test, predictions))

In [12]:
if __name__ == "__main__":
    main()
    

initial b_2: [[-0.05849051  0.13005025  0.34326399  0.12310553  0.19861653 -0.28221634
  -0.43211352 -0.2805326  -0.09144563 -0.34479613]]
initial w_2: [[ 0.1017466  -0.14100496  0.22776536 ...  0.12002991  0.4498872
   0.35015966]
 [ 0.47496679 -0.05990181 -0.17118605 ... -0.38828319 -0.21033889
  -0.24321102]
 [ 0.05101367  0.26990017 -0.47416377 ... -0.05884594 -0.00789025
   0.09935907]
 ...
 [-0.40242027 -0.20120645 -0.20678906 ... -0.47687336 -0.18057245
   0.39457007]
 [-0.13412645  0.00874054  0.02862964 ... -0.3952518  -0.29190923
   0.30038228]
 [ 0.40319524 -0.07172258 -0.09330041 ...  0.04341981 -0.40878256
  -0.07619746]]
epoch idx: 0
accuracy for epoch idx 0 is: 0.3781
epoch idx: 1
accuracy for epoch idx 1 is: 0.4076
epoch idx: 2
accuracy for epoch idx 2 is: 0.4185
epoch idx: 3
accuracy for epoch idx 3 is: 0.4285
epoch idx: 4
accuracy for epoch idx 4 is: 0.4325
epoch idx: 5
accuracy for epoch idx 5 is: 0.4356
epoch idx: 6
accuracy for epoch idx 6 is: 0.4409
epoch idx: 7
a